In [66]:
# Imports
import csv
from bs4 import BeautifulSoup, Tag
from datetime import datetime
import sqlite3
import requests
import pandas as pd

In [67]:
# Conectando com o db
conn = sqlite3.connect('../../Dados/database/valorantmatches.db')
c = conn.cursor()

In [68]:
# Função para pegar scores round por round de um time

def get_scores(map_index, team_index, team_list):
  round_scores_list = []
  vlr_round_by_round = all_maps_status_list[map_index].find_all('div', class_='vlr-rounds-row')[0]
  vlr_round_by_round_col = vlr_round_by_round.find_all(class_='vlr-rounds-row-col')
  
  for round in vlr_round_by_round_col:
    round_scores = round.get('title')
    round_scores_list.append(round_scores)
    filtered_round_scores_list = [score for score in round_scores_list if score is not None]
  for score in filtered_round_scores_list:
    numbers = score.split('-')
    if len(numbers) == 2:
      team_list.append(int(numbers[team_index]))
    else:
      team_list.append(0)

# Função para pegar o side de cada time

def get_side(map_index):
  first_side_winner = ''
  vlr_round_by_round = all_maps_status_list[map_index].find_all('div', class_='vlr-rounds-row')[0]
  vlr_round_by_round_col = vlr_round_by_round.find_all(class_='vlr-rounds-row-col')
  first_side_div = vlr_round_by_round_col[1].find('div', class_='mod-win')
  if first_side_div is not None:
    first_side_value = first_side_div.get('class')
    if 'mod-t' in first_side_value:
      first_side_winner = 'Attack'
      return first_side_winner
    if 'mod-ct' in first_side_value:
      first_side_winner = 'Defend'
      return first_side_winner
  else:
    return first_side_winner

#Função para pegar diferentes status dos players

def get_player_few_status(row_player_status, def_status_list, def_class_name, def_index ):
  get_status = row_player_status.find_all('td', class_=f'{def_class_name}')[def_index].find_all('span', class_='side')
  for value in get_status:
    def_status_list.append(value.text)

#Função para selecionar a tabela do time

def get_team_table(map_index, table_index):
  table_team_players = all_maps_status_list[map_index].find_all('table', class_='wf-table-inset mod-overview')[table_index]
  return table_team_players

#Função para aumentar o len de status com menos de 3 valores

def get_more_len(stats_list):
  while len(stats_list) < 3:
    stats_list.append('')

#Função para substituir valores indesejados das listas

def get_undesirable_values(stats_list):
    replaced_list = []
    for item in stats_list:
        if item == '\xa0':
            replaced_list.append(0)
        else:
            replaced_list.append(float(item.strip('%').replace(',', '')))
    return replaced_list



In [69]:
NAME_FIXES = {
    'Guangzhou': 'Bilibili Gaming',
    'Movistar': 'Movistar KOI',
    'JD Mall': 'JDG Esports',
    'KRÜ': 'KRU Esports',
    'Nongshim': 'Nongshim RedForce',
    'Dragon Ranger': 'Dragon Ranger Gaming',
    'Titan Esports': 'Titan Esports Club',    
    'FunPlus': 'FunPlus Phoenix',
}

#Função para consertar os nomes dos times
def get_undesirable_names(team_name):
    for key, value in NAME_FIXES.items():
        if key in team_name:
            return value
    return team_name

In [70]:
VCT_URL_list = [
    'https://www.vlr.gg/event/matches/2282/valorant-masters-2025-toronto/?series_id=all',
    'https://www.vlr.gg/event/matches/2380/champions-tour-2025-emea-stage-1/?series_id=all',
    'https://www.vlr.gg/event/matches/2347/champions-tour-2025-americas-stage-1/?series_id=all',
    'https://www.vlr.gg/event/matches/2379/champions-tour-2025-pacific-stage-1/?series_id=all',
    'https://www.vlr.gg/event/matches/2359/champions-tour-2025-china-stage-1/?series_id=all',
    'https://www.vlr.gg/event/matches/2281/champions-tour-2025-masters-bangkok/?series_id=all',
    'https://www.vlr.gg/event/matches/2276/champions-tour-2025-emea-kickoff/?series_id=all',
    'https://www.vlr.gg/event/matches/2274/champions-tour-2025-americas-kickoff/?series_id=all',
    'https://www.vlr.gg/event/matches/2277/champions-tour-2025-pacific-kickoff/?series_id=all',
    'https://www.vlr.gg/event/matches/2275/champions-tour-2025-china-kickoff/?series_id=all',
]

In [71]:
# lists que passaram pelo scrapping
tournament_url_list = VCT_URL_list

In [72]:
# Lists de armazenamento dos dados
general_data_list = []
players_data_list = []
rounds_data_list = []

In [73]:
for tournament_url in tournament_url_list:

  # URL do projeto e configs
  url = tournament_url
  page = requests.get(url)
  soup = BeautifulSoup(page.content, 'html.parser')
  link_list = []
  row_links = soup.find_all('a', class_='wf-module-item')

  # Filtrando URLs dos games
  for row in row_links:
    match_status = row.find('div', class_='match-item-eta').find('div', class_='ml-status').text
    if match_status == 'Completed':
      link = row.get('href')
      link_list.append(link)

  # Pega urls do db
  urls_to_remove = ['/450589/team-international-vs-team-thailand-champions-tour-2025-masters-bangkok-main-event']
  match_urls = set()
  c.execute("SELECT url FROM general_match_data")
  db_urls =[link[0] for link in c.fetchall()]
  for url in db_urls:
    match_urls.add(url)

  for url in link_list:
    if f'https://www.vlr.gg{url}' in match_urls:
      urls_to_remove.append(url)

  # Remove URLs dos games que já estão no arquivo
  for url in urls_to_remove:
    if url in link_list:
      link_list.remove(url)

  # Loop para scrapping dos dados
  for link in link_list:
    link_url = f'https://www.vlr.gg{link}'
    link_page = requests.get(link_url)
    link_soup = BeautifulSoup(link_page.content, 'html.parser')
    print(link_url)

    # Pegando nome do campeonato e a fase em que ele está na partida jogada
    row_header = link_soup.find(class_='match-header-super')
    row_header_event = row_header.find(class_='match-header-event')
    row_header_texts = row_header_event.find('div')
    row_tournament_name = row_header_texts.find('div').text.strip()
    row_tournament_phase = row_header_texts.find(class_="match-header-event-series").text.strip()
    
    tournament_name = row_tournament_name.replace('\n', '').replace('\t', '')
    tournament_phase = row_tournament_phase.replace('\n', '').replace('\t', '')

    # Pegando a data e horario da partida em UTC
    row_date = row_header.find(class_='moment-tz-convert')
    if isinstance(row_date, Tag) and row_date.get('data-utc-ts'):
      date_utc = row_date.get('data-utc-ts')
    date_object = datetime.strptime(date_utc, '%Y-%m-%d %H:%M:%S')
    year = date_object.year

    if len(row_header.find_all('div')) >= 9:
      tournament_patch = row_header.find_all('div')[8].text.strip()
    else:
      tournament_patch = ''

    # Pegando nomes dos times
    row_header_score_div = link_soup.find(class_='match-header-vs')
    row_header_team_1_div = row_header_score_div.find(class_='match-header-link-name mod-1')
    row_header_team_2_div = row_header_score_div.find(class_='match-header-link-name mod-2')

    team_1_value = row_header_team_1_div.find(class_='wf-title-med').text.strip()
    team_2_value = row_header_team_2_div.find(class_='wf-title-med').text.strip()

    # Corrige os nomes dos times
    team_1 = get_undesirable_names(team_1_value)
    team_2 = get_undesirable_names(team_2_value)
    print(f'{team_1} x {team_2}')

    # Pegando resultado final
    row_header_final_score_div = row_header_score_div.find(class_='js-spoiler')

    score_team_1 = row_header_score_div.find_all('span')[0].text.strip()
    score_team_2 = row_header_score_div.find_all('span')[2].text.strip()

    # Pegando formato da série
    format = row_header_score_div.find_all(class_='match-header-vs-note')[1].text.strip()

    # Pegando picks e bans
    if link_soup.find('div', class_='match-header-note') is not None:

      header_picks_bans = link_soup.find_all(class_='match-header-note')

      if len(header_picks_bans) > 1:
        picks_bans_split = header_picks_bans[1].text.split(';')
      else:
        picks_bans_split = header_picks_bans[0].text.split(';')
      
      picks_bans = [map.strip() for map in picks_bans_split]

      team_that_ban_first = picks_bans[0].split()[0]
      map_remain_string = picks_bans.pop()
      map_remain_filter = map_remain_string.split()
      map_remain_filter.pop()
      map_remain = ''.join(map_remain_filter)
      maps_banned_picked = [' '.join(item.split()[2:]) for item in picks_bans]
      maps_banned_picked.append(map_remain)
    else:
      team_that_ban_first = ''
      maps_banned_picked = ['','','','','','','']

    # Pegando ID da partida
    match_id = link_soup.find(class_='vm-stats').get('data-match-id')

    # Pegando status de todos os mapas.
    i = 0
    all_maps_status_list = []
    number_of_games = len(link_soup.find_all(class_='vm-stats-game'))

    while i< number_of_games:
      map_status = link_soup.find_all(class_='vm-stats-game')[i]
      all_maps_status_list.append(map_status)
      i+=1

    # Setando picks e bans da série
    if len(maps_banned_picked) == 7:
      if format == 'Bo5':
        ban_1 = maps_banned_picked[0]
        ban_2 = maps_banned_picked[1]
        ban_3 = ''
        ban_4 = ''
        pick_1 = maps_banned_picked[2]
        pick_2 = maps_banned_picked[3]
        pick_3 =  maps_banned_picked[4]
        pick_4 = maps_banned_picked[5]
        decider = maps_banned_picked[6]
        picks_list = [maps_banned_picked[2],maps_banned_picked[3],maps_banned_picked[4],maps_banned_picked[5],maps_banned_picked[6]]

      if format == 'Bo3':
        ban_1 = maps_banned_picked[0]
        ban_2 = maps_banned_picked[1]
        ban_3 = maps_banned_picked[4]
        ban_4 = maps_banned_picked[5]
        pick_1 = maps_banned_picked[2]
        pick_2 = maps_banned_picked[3]
        pick_3 =  ''
        pick_4 = ''
        decider = maps_banned_picked[6]
        picks_list = [maps_banned_picked[2],maps_banned_picked[3],maps_banned_picked[6]]

      if format == 'Bo1':
        ban_1 = ''
        ban_2 = ''
        ban_3 = ''
        ban_4 = ''
        pick_1 = ''
        pick_2 = ''
        pick_3 =  ''
        pick_4 = ''
        decider = maps_banned_picked[6]
        picks_list = [maps_banned_picked[6]]

    else:
      if format == 'Bo1':
        ban_1 = ''
        ban_2 = ''
        ban_3 = ''
        ban_4 = ''
        pick_1 = ''
        pick_2 = ''
        pick_3 =  ''
        pick_4 = ''
        decider = ''
        picks_list = [all_maps_status_list[0].get('data-game-id')]
      else:
        if format == 'Bo3':
          ban_1 = ''
          ban_2 = ''
          ban_3 = ''
          ban_4 = ''
          pick_1 = maps_banned_picked[0]
          pick_2 = maps_banned_picked[1]
          pick_3 =  ''
          pick_4 = ''
          decider = maps_banned_picked[2]
          picks_list = [maps_banned_picked[0],maps_banned_picked[1],maps_banned_picked[2]]

        if format == 'Bo5':
          ban_1 = ''
          ban_2 = ''
          ban_3 = ''
          ban_4 = ''
          pick_1 = maps_banned_picked[0]
          pick_2 = maps_banned_picked[1]
          pick_3 =  maps_banned_picked[2]
          pick_4 = maps_banned_picked[3]
          decider = maps_banned_picked[4]
          picks_list = [maps_banned_picked[0],maps_banned_picked[1],maps_banned_picked[2],maps_banned_picked[3],maps_banned_picked[4]]

    j = 0
    l = 0

    number_of_maps = len(all_maps_status_list)

    while j < number_of_maps:
      t1_round_by_round = []
      t2_round_by_round = []
      map_id = all_maps_status_list[j].get('data-game-id')

      if map_id == 'all':
        map_duration = ''
        map_name = 'all'
        first_side_t1 = ''
        first_side_t2 = ''
        t1_round_by_round = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        t2_round_by_round = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        map_t1_final_score = score_team_1
        map_t2_final_score = score_team_2
        
      else:
        map_t1_final_score = all_maps_status_list[j].find('div', class_='vm-stats-game-header').find('div', class_='team').find('div', class_='score').text.strip()
        map_t2_final_score = all_maps_status_list[j].find('div', class_='vm-stats-game-header').find('div', class_='team mod-right').find('div', class_='score').text.strip()
        map_duration = all_maps_status_list[j].find('div', class_='map').find('div', class_= 'map-duration ge-text-light').text.strip()
        map_name = picks_list[l]

        # Pegando o side inicial de cada time
        get_scores(j, 0, t1_round_by_round)
        get_scores(j, 1, t2_round_by_round)
        first_side_winner = get_side(j)
        first_side_loser = ''

        if first_side_winner == '':
          first_side_t1 = ''
          first_side_t2 = ''
        if first_side_winner == 'Attack':
          first_side_loser = 'Defend'
        else:
          first_side_loser = 'Attack'
        
        if t1_round_by_round[0]> t2_round_by_round[0]:
          first_side_t1 = first_side_winner
          first_side_t2 = first_side_loser
        else:
          first_side_t1 = first_side_loser
          first_side_t2 = first_side_winner
        l+=1
      
      team_t1_tournament_info = (link_url, tournament_name, year, tournament_phase, date_utc, team_1,team_2,f'{team_1}x{team_2}', match_id, map_id, tournament_patch,
                                    team_1, team_that_ban_first, ban_1, ban_2, ban_3, ban_4, pick_1, pick_2, pick_3, pick_4, decider, map_name, map_t1_final_score)
        
      team_t2_tournament_info = (link_url, tournament_name, year, tournament_phase, date_utc, team_1,team_2,f'{team_1}x{team_2}', match_id, map_id, tournament_patch,
                                    team_2, team_that_ban_first, ban_1, ban_2, ban_3, ban_4, pick_1, pick_2, pick_3, pick_4, decider, map_name, map_t2_final_score)

      # Pegando os rounds dos mapas
      rounds_t1_tournament_info = tuple([match_id, map_id, team_1, first_side_t1, map_name, map_duration, map_t1_final_score] + t1_round_by_round)
      rounds_t2_tournament_info = tuple([match_id, map_id, team_2, first_side_t2, map_name, map_duration, map_t2_final_score] + t2_round_by_round)
      
      # Adicionando os valores gerais na list 
      general_data_list.append(team_t1_tournament_info)
      general_data_list.append(team_t2_tournament_info)

      # Adicionando os valores dos rounds na list
      rounds_data_list.append(rounds_t1_tournament_info)
      rounds_data_list.append(rounds_t2_tournament_info)

      # Pegando status do time 1

      table_team_1_players = get_team_table(j, 0)
      table_team_2_players = get_team_table(j, 1)

      k = 0
      while k <= 4:

        tr_player_t1_status = table_team_1_players.find_all('tr')[k+1]

        player_t1_initial_informations = tr_player_t1_status.find('td', class_='mod-player')

        # Nome do player
        player_t1_name = player_t1_initial_informations.find('div', class_='text-of').text.strip()

        # País do player
        player_t1_country =  player_t1_initial_informations.find(class_='flag').get('title')

        # Agente jogado pelo player
        mod_agents_td = tr_player_t1_status.find('td', class_='mod-agents')
        img_tag = mod_agents_td.find('img') if mod_agents_td else None
        player_t1_agent = img_tag.get('title') if img_tag else ''

        # [R] Rating do player
        player_t1_rating = []
        get_player_few_status(tr_player_t1_status, player_t1_rating, 'mod-stat', 0)
        player_t1_rating = get_undesirable_values(player_t1_rating)

        # [ACS]
        player_t1_acs = []
        get_player_few_status(tr_player_t1_status, player_t1_acs, 'mod-stat', 1)
        player_t1_acs = get_undesirable_values(player_t1_acs)

        # [K] Kills do player mod-vlr-kills
        player_t1_k = []
        get_player_few_status(tr_player_t1_status, player_t1_k, 'mod-vlr-kills', 0)
        if len(player_t1_k) < 3:
          get_more_len(player_t1_k)

        # [D] Deaths do player
        player_t1_d = []
        get_player_few_status(tr_player_t1_status, player_t1_d, 'mod-vlr-deaths', 0)
        if len(player_t1_d) < 3:
          get_more_len(player_t1_d)

        # [A] Assists do player
        player_t1_a = []
        get_player_few_status(tr_player_t1_status, player_t1_a, 'mod-vlr-assists', 0)
        if len(player_t1_a) < 3:
          get_more_len(player_t1_a)

        # [KD+-] KD diff do player
        player_t1_kd = []
        get_player_few_status(tr_player_t1_status, player_t1_kd, 'mod-kd-diff', 0)
        player_t1_kd = get_undesirable_values(player_t1_kd)

        # [KAST] KAST do player
        player_t1_kast = []
        get_player_few_status(tr_player_t1_status, player_t1_kast, 'mod-stat', 6)
        player_t1_kast = get_undesirable_values(player_t1_kast)

        # [ADR] ADR do player
        player_t1_adr = []
        get_player_few_status(tr_player_t1_status, player_t1_adr, 'mod-stat', 7)
        player_t1_adr = get_undesirable_values(player_t1_adr)

        # [HS%] HS% do player
        player_t1_hs = []
        get_player_few_status(tr_player_t1_status, player_t1_hs, 'mod-stat', 8)
        player_t1_hs = get_undesirable_values(player_t1_hs)

        # [FB] FB do player
        player_t1_fb = []
        get_player_few_status(tr_player_t1_status, player_t1_fb, 'mod-fb', 0)
        player_t1_fb = get_undesirable_values(player_t1_fb)

        # [FD] FD do player
        player_t1_fd = []
        get_player_few_status(tr_player_t1_status, player_t1_fd, 'mod-fd', 0)
        player_t1_fd = get_undesirable_values(player_t1_fd)


        # [FK+-] FK diff do player
        player_t1_fk = []
        get_player_few_status(tr_player_t1_status, player_t1_fk, 'mod-fk-diff', 0)
        player_t1_fk = get_undesirable_values(player_t1_fk)

        # Organizando os valores coletados do scrapping        
        player_t1_overall_stats = (player_t1_rating[0], player_t1_acs[0], player_t1_k[0], player_t1_d[0], player_t1_a[0], player_t1_kd[0], player_t1_kast[0], 
                                   player_t1_adr[0], player_t1_hs[0], player_t1_fb[0], player_t1_fd[0], player_t1_fk[0])
        player_t1_attack_stats = (player_t1_rating[1], player_t1_acs[1], player_t1_k[1], player_t1_d[1], player_t1_a[1], player_t1_kd[1], player_t1_kast[1],
                                   player_t1_adr[1], player_t1_hs[1], player_t1_fb[1], player_t1_fd[1], player_t1_fk[1])
        player_t1_defend_stats = (player_t1_rating[2], player_t1_acs[2], player_t1_k[2], player_t1_d[2], player_t1_a[2], player_t1_kd[2], player_t1_kast[2], 
                                  player_t1_adr[2], player_t1_hs[2], player_t1_fb[2], player_t1_fd[2], player_t1_fk[2])

        if map_id == 'all':
          player_t1_match_info = (match_id, map_id, player_t1_name, player_t1_country, team_1, '', '')
        else:
          player_t1_match_info = (match_id, map_id, player_t1_name, player_t1_country, team_1, player_t1_agent, map_name)

        #Adicionando dados do player t1 para a player list
        players_data_list.append(player_t1_match_info + player_t1_overall_stats + player_t1_attack_stats + player_t1_defend_stats)
        k+=1

      m = 0
      while m <= 4:

        tr_player_t2_status = table_team_2_players.find_all('tr')[m+1]

        player_t2_initial_informations = tr_player_t2_status.find('td', class_='mod-player')

        # Nome do player
        player_t2_name = player_t2_initial_informations.find('div', class_='text-of').text.strip()

        # País do player
        player_t2_country =  player_t2_initial_informations.find(class_='flag').get('title')

        # Agente jogado pelo player
        mod_agents_td = tr_player_t2_status.find('td', class_='mod-agents')
        img_tag = mod_agents_td.find('img') if mod_agents_td else None
        player_t2_agent = img_tag.get('title') if img_tag else ''

        # [R] Rating do player
        player_t2_rating = []
        get_player_few_status(tr_player_t2_status, player_t2_rating, 'mod-stat', 0)
        player_t2_rating = get_undesirable_values(player_t2_rating)

        # [ACS]
        player_t2_acs = []
        get_player_few_status(tr_player_t2_status, player_t2_acs, 'mod-stat', 1)
        player_t2_acs = get_undesirable_values(player_t2_acs)

        # [K] Kills do player mod-vlr-kills
        player_t2_k = []
        get_player_few_status(tr_player_t2_status, player_t2_k, 'mod-vlr-kills', 0)
        if len(player_t2_k) < 3:
          get_more_len(player_t2_k)

        # [D] Deaths do player
        player_t2_d = []
        get_player_few_status(tr_player_t2_status, player_t2_d, 'mod-vlr-deaths', 0)
        if len(player_t2_d) < 3:
          get_more_len(player_t2_d)

        # [A] Assists do player
        player_t2_a = []
        get_player_few_status(tr_player_t2_status, player_t2_a, 'mod-vlr-assists', 0)
        if len(player_t2_a) < 3:
          get_more_len(player_t2_a)

        # [KD+-] KD diff do player
        player_t2_kd = []
        get_player_few_status(tr_player_t2_status, player_t2_kd, 'mod-kd-diff', 0)
        player_t2_kd = get_undesirable_values(player_t2_kd)

        # [KAST] KAST do player
        player_t2_kast = []
        get_player_few_status(tr_player_t2_status, player_t2_kast, 'mod-stat', 6)
        player_t2_kast = get_undesirable_values(player_t2_kast)

        # [ADR] ADR do player
        player_t2_adr = []
        get_player_few_status(tr_player_t2_status, player_t2_adr, 'mod-stat', 7)
        player_t2_adr = get_undesirable_values(player_t2_adr)

        # [HS%] HS% do player
        player_t2_hs = []
        get_player_few_status(tr_player_t2_status, player_t2_hs, 'mod-stat', 8)
        player_t2_hs = get_undesirable_values(player_t2_hs)

        # [FB] FB do player
        player_t2_fb = []
        get_player_few_status(tr_player_t2_status, player_t2_fb, 'mod-fb', 0)
        player_t2_fb = get_undesirable_values(player_t2_fb)

        # [FD] FD do player
        player_t2_fd = []
        get_player_few_status(tr_player_t2_status, player_t2_fd, 'mod-fd', 0)
        player_t2_fd = get_undesirable_values(player_t2_fd)

        # [FK+-] FK diff do player
        player_t2_fk = []
        get_player_few_status(tr_player_t2_status, player_t2_fk, 'mod-fk-diff', 0)
        player_t2_fk = get_undesirable_values(player_t2_fk)

        # Organizando os valores coletados do scrapping
        player_t2_overall_stats = (player_t2_rating[0], player_t2_acs[0], player_t2_k[0], player_t2_d[0], player_t2_a[0], player_t2_kd[0], player_t2_kast[0], 
                                   player_t2_adr[0], player_t2_hs[0], player_t2_fb[0], player_t2_fd[0], player_t2_fk[0])
        player_t2_attack_stats = (player_t2_rating[1], player_t2_acs[1], player_t2_k[1], player_t2_d[1], player_t2_a[1], player_t2_kd[1], player_t2_kast[1],
                                   player_t2_adr[1], player_t2_hs[1], player_t2_fb[1], player_t2_fd[1], player_t2_fk[1])
        player_t2_defend_stats = (player_t2_rating[2], player_t2_acs[2], player_t2_k[2], player_t2_d[2], player_t2_a[2], player_t2_kd[2], player_t2_kast[2], 
                                  player_t2_adr[2], player_t2_hs[2], player_t2_fb[2], player_t2_fd[2], player_t2_fk[2])

        if map_id == 'all':
          player_t2_match_info = (match_id, map_id, player_t2_name, player_t2_country, team_2, '', '')
        else:
          player_t2_match_info = (match_id, map_id, player_t2_name, player_t2_country, team_2, player_t2_agent, map_name)
        
        # Adicionando dados do player t1 para a player list
        players_data_list.append(player_t2_match_info + player_t2_overall_stats + player_t2_attack_stats + player_t2_defend_stats)

        m+=1

      j+=1

https://www.vlr.gg/459826/team-vitality-vs-giantx-champions-tour-2025-emea-stage-1-w1
Team Vitality x GIANTX
https://www.vlr.gg/459827/bbl-esports-vs-karmine-corp-champions-tour-2025-emea-stage-1-w1
BBL Esports x Karmine Corp
https://www.vlr.gg/459828/team-heretics-vs-apeks-champions-tour-2025-emea-stage-1-w1
Team Heretics x Apeks
https://www.vlr.gg/459829/fut-esports-vs-koi-champions-tour-2025-emea-stage-1-w1
FUT Esports x Movistar KOI
https://www.vlr.gg/459830/fnatic-vs-natus-vincere-champions-tour-2025-emea-stage-1-w1
FNATIC x Natus Vincere
https://www.vlr.gg/459831/team-liquid-vs-gentle-mates-champions-tour-2025-emea-stage-1-w1
Team Liquid x Gentle Mates
https://www.vlr.gg/459832/team-liquid-vs-bbl-esports-champions-tour-2025-emea-stage-1-w2
Team Liquid x BBL Esports
https://www.vlr.gg/459833/apeks-vs-natus-vincere-champions-tour-2025-emea-stage-1-w2
Apeks x Natus Vincere
https://www.vlr.gg/459834/fut-esports-vs-gentle-mates-champions-tour-2025-emea-stage-1-w2
FUT Esports x Gentle 

In [74]:
#Criando dfs

valorant_general_data_df = pd.DataFrame(general_data_list, columns=['url','tournament','year','tournament_phase','date','team_1','team_2','match','match_id','map_id','patch',
                                                                    'team_name','first_to_ban','ban1','ban2','ban3','ban4','pick1','pick2','pick3','pick4','decider','map_name',
                                                                    'result'])

valorant_players_data_df = pd.DataFrame(players_data_list, columns=['match_id','map_id','player_name','player_country','team_name','agent_name','map_name','overall_rating',
                                                                    'overall_ACS','overall_K','overall_D','overall_A','overall_KD_avr','overall_KAST','overall_ADR',
                                                                    'overall_HS_percent','overall_FK','overall_FD','overall_F_avr','attack_rating','attack_ACS','attack_K',
                                                                    'attack_D','attack_A','attack_KD_avr','attack_KAST','attack_ADR','attack_HS_percent','attack_FK',
                                                                    'attack_FD','attack_F_avr','defend_rating','defend_ACS','defend_K','defend_D','defend_A','defend_KD_avr',
                                                                    'defend_KAST','defend_ADR','defend_HS_percent','defend_FK','defend_FD','defend_F_avr'])

valorant_rounds_data_df = pd.DataFrame(rounds_data_list, columns=['match_id','map_id','team_name','first_side','map_name','game_length','result','r1','r2','r3','r4','r5','r6',
                                                                  'r7','r8','r9','r10','r11','r12','r13','r14','r15','r16','r17','r18','r19','r20','r21','r22','r23','r24'])


In [75]:
#Adicionar os dados em csv
#valorant_general_data_df.to_csv('../../Dados/2025/general_data.csv', index=False)
#valorant_players_data_df.to_csv('../../Dados/2025/players_data.csv', index=False)
#valorant_rounds_data_df.to_csv('../../Dados/2025/rounds_data.csv', index=False)
#print('Dados adicionados na df com sucesso')

In [76]:
#Adicionar dados das dfs no db
valorant_general_data_df.to_sql('general_match_data', conn, if_exists='append', index=False)
valorant_players_data_df.to_sql('player_match_data', conn, if_exists='append', index=False)
valorant_rounds_data_df.to_sql('rounds_match_data', conn, if_exists='append', index=False)
print('Dados adicionados no db com sucesso')

Dados adicionados no db com sucesso
